In [1]:
import numpy as np
import pandas as pd
import os

from common import OUTPUTPATH, set_seed
from models import rule_fit_pcc
import data

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)


<ipython-input-1-aa8db4ec1bfc>:9: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


This notebook refit the models in the all dataset

Fit all data takes ~10 mim under 2.6 GHz 6-Core Intel Core i7

In [2]:
set_seed(1000)
rule_fit_pcc.fit(data.x1, data.y)

In [3]:
fitted_model = rule_fit_pcc.fitted_
mophology = ['Sphere', 'Worm', 'Vesicle', 'Other']
for i in range(len(fitted_model)):
    print(mophology[i], 'Cs:', fitted_model[i].model.Cs)

Sphere Cs: [1]
Worm Cs: [1]
Vesicle Cs: [1]
Other Cs: [32]


Get the individual morphology rules

In [4]:
all_columns = data.predictors + data.targets
rule_sphere = rule_fit_pcc.fitted_[0].get_rules(all_columns)[['rule','type','coef','importance']]
rule_sphere.to_csv(os.path.join(OUTPUTPATH, 'Sphere_rules.csv'), index=False)
rule_sphere

,rule,type,coef,importance
295,dp_core> 209.5 & conc> 16.1 & mon_corona_psa> 65.0 & corona_mv_total> 42.9745 & dp_corona<= 95.0 & mon_core_psa> 36.415,rule,-3.807906,1.211516
1345,dp_corona> 26.5 & corona_mv_total> 54.5619 & salt<= 0.2 & conc<= 21.25 & mon_core_mw> 143.9765 & mon_corona_mw<= 287.115 & conc<= 16.5,rule,2.655729,1.194534
626,mon_corona_apol<= 28.3883 & ratio_vol<= 0.2213 & clogp_corona<= 0.73 & clogp_core<= 1.4 & mon_core_psa<= 47.1448 & dp_corona<= 64.5,rule,-2.554682,0.980607
551,dp_corona> 27.5 & mon_corona_apol<= 58.2307 & clogp_core<= 1.18 & dp_corona<= 62.5,rule,-1.460056,0.730022
187,core_mw_total> 5330.96 & conc> 6.25 & charged<= 0.5 & mon_core_mw<= 144.4899,rule,-1.497569,0.714774
546,mon_core_mw<= 145.3096 & corona_mw_total<= 12760.5298 & conc> 6.25 & ratio_vol<= 0.3104,rule,-1.333342,0.664663
230,ratio_mass<= 0.325 & mon_core_psa<= 46.5831 & corona_mv_total> 88.7037 & clogp_core<= 1.37,rule,1.938582,0.607055
827,corona_mw_total<= 15743.5601 & conc<= 21.5 & corona_mw_total<= 9204.0249 & mon_core_mw<= 144.8274 & charged> 0.5,rule,2.142509,0.573044
774,dp_corona<= 61.0 & mon_corona_apol<= 23.6109 & clogp_core<= 1.37,rule,-1.106356,0.532091
276,conc<= 14.25 & clogp_core> 1.105,rule,1.724554,0.522026


In [5]:
rule_worm = rule_fit_pcc.fitted_[1].get_rules(all_columns)[['rule','type','coef','importance']]
rule_worm.to_csv(os.path.join(OUTPUTPATH, 'Worm_rules.csv'), index=False)
rule_worm

,rule,type,coef,importance
867,ratio_vol> 0.2138 & ratio_vol> 0.2161,rule,-1.774555,0.882646
648,clogp_corona<= 0.25 & mon_core_psa<= 46.7805 & conc<= 11.25 & clogp_core<= 1.37,rule,-1.626521,0.749633
1433,sphere<= 0.5 & mon_core_psa<= 46.7805 & ratio_mass<= 0.2346 & core_mw_total> 32345.96,rule,-1.639862,0.655945
308,mon_core_mw> 144.0775 & sphere> 0.5,rule,-1.276363,0.637751
206,dp_corona<= 77.5 & ratio_vol<= 0.2379 & mon_core_psa<= 46.7729 & dp_corona<= 61.0,rule,-1.239020,0.603698
134,mon_core_psa> 46.35 & core_mv_total> 133.3937 & dp_core<= 299.5 & clogp_corona<= 0.73,rule,1.196749,0.590744
440,conc<= 23.75 & ratio_vol> 0.1529 & corona_mw_total> 4599.625,rule,-1.007952,0.498807
947,corona_mw_total<= 5985.855 & clogp_core<= 1.13 & mon_core_psa<= 46.7501 & ratio_mass> 0.1459 & corona_mw_total> 2972.075,rule,1.693711,0.463745
1176,ratio_vol> 0.2783 & mon_core_mw> 144.0775 & corona_mw_total> 5897.5901 & sphere> 0.5,rule,-1.216186,0.425577
97,ratio_vol> 0.2138 & sphere<= 0.5 & clogp_core> 0.665,rule,1.074647,0.416122


In [6]:
rule_vesicle = rule_fit_pcc.fitted_[2].get_rules(all_columns)[['rule','type','coef','importance']]
rule_vesicle.to_csv(os.path.join(OUTPUTPATH, 'Vesicle_rules.csv'), index=False)
rule_vesicle

,rule,type,coef,importance
812,ratio_vol<= 0.2604 & mon_corona_mv> 0.7838 & ratio_mass> 0.0443 & sphere<= 0.5 & temp> 53.0 & corona_mw_total<= 14063.4102 & ratio_mass<= 0.3571 & mon_core_apol> 20.5139 & dp_corona> 30.0,rule,2.672133,1.196556
429,mon_core_psa<= 46.5755 & sphere> 0.5 & conc> 7.5,rule,-1.597402,0.796776
378,clogp_core<= 1.18 & ratio_vol<= 0.2422 & mon_corona_psa> 65.0 & corona_mw_total<= 12760.5298 & core_mv_total<= 1362.75,rule,1.761069,0.761525
1135,corona_mw_total> 2502.955 & core_mw_total<= 57830.3594 & ratio_mass<= 0.2483 & core_mv_total> 258.0855 & dp_corona> 29.5,rule,1.331125,0.569161
551,corona_mw_total> 3982.275 & corona_mw_total<= 5806.375 & ratio_vol<= 0.3723,rule,1.617735,0.559271
618,ratio_mass<= 0.2777 & sphere<= 0.5 & corona_mw_total> 4260.0101 & mon_core_psa<= 46.5831 & mon_core_mw> 142.9185 & ratio_vol> 0.08,rule,1.191721,0.538364
571,mon_core_mw<= 144.4899 & mon_corona_psa> 65.0 & clogp_corona> -5.23 & mon_core_psa<= 47.1448 & ratio_mass<= 0.2467 & conc> 11.25,rule,1.502053,0.477890
332,conc<= 22.75 & dp_corona<= 53.5,rule,-0.932582,0.453496
71,mon_corona_psa<= 67.2702 & conc> 6.25 & ph> 3.85,rule,-0.912843,0.440065
498,mon_core_mw> 142.8175 & ratio_vol<= 0.2662 & clogp_core<= 1.35 & mon_corona_psa> 65.0 & clogp_corona> -5.23,rule,0.825321,0.374421


In [7]:
rule_other = rule_fit_pcc.fitted_[3].get_rules(all_columns)[['rule','type','coef','importance']]
rule_other.to_csv(os.path.join(OUTPUTPATH, 'Other_rules.csv'), index=False)
rule_other

,rule,type,coef,importance
437,sphere> 0.5,rule,-2.231290,1.101420
23,sphere,linear,-2.141922,1.067999
762,salt<= 0.15 & clogp_corona<= 0.73 & sphere<= 0.5 & clogp_core<= 1.4 & corona_mv_total<= 107.5755,rule,-1.912289,0.947431
402,sphere<= 0.5 & mon_corona_psa<= 80.73 & corona_mv_total> 117.3243,rule,4.644186,0.656650
407,salt<= 0.2 & sphere<= 0.5 & conc> 18.75 & ratio_vol> 0.1218 & clogp_corona> 0.125,rule,4.594389,0.649609
854,sphere<= 0.5 & mon_corona_psa<= 47.97 & mon_corona_psa> 23.305,rule,5.284313,0.581153
790,clogp_core<= 1.1121 & salt<= 0.2,rule,-0.858438,0.401246
680,core_mv_total> 70.9875 & clogp_corona<= 0.125 & mon_corona_mv> 0.7838 & salt<= 0.2 & mon_core_psa<= 46.5603,rule,-0.790664,0.311320
489,corona_mw_total<= 14338.48 & core_mw_total> 8580.3198,rule,-0.938138,0.303058
897,conc> 17.5 & clogp_corona<= 0.73 & sphere<= 0.5 & mon_core_psa<= 46.5755 & ratio_vol> 0.1218 & mon_corona_mv<= 0.7838,rule,2.623818,0.288560


Get the Top5 morphology rules

In [8]:
# get top 5:
sphere_top5_rule = rule_sphere.iloc[:5, ]
worm_top5_rule = rule_worm.iloc[:5, ]
vesicle_top5_rule = rule_vesicle.iloc[:5, ]

sphere_top5_rule.reset_index(drop=True, inplace=True)
worm_top5_rule.reset_index(drop=True, inplace=True)
vesicle_top5_rule.reset_index(drop=True, inplace=True)

top5_rule = pd.concat([sphere_top5_rule, worm_top5_rule, vesicle_top5_rule], axis=1)
upper_columns = ['Sphere', 'Worm', 'Vesicle']
lower_columns = ['rule', 'type', 'coef', 'importance']

top5_rule.columns = pd.MultiIndex.from_product([upper_columns, lower_columns], names=['Phase', 'Property'])
top5_rule.to_csv(os.path.join(OUTPUTPATH, 'Combine_Top5_Rules.csv'), index=False)
top5_rule

Phase                                                                                                                                     Sphere  \
Property                                                                                                                                    rule   
0         dp_core> 209.5 & conc> 16.1 & mon_corona_psa> 65.0 & corona_mv_total> 42.9745 & dp_corona<= 95.0 & mon_core_psa> 36.415                  
1         dp_corona> 26.5 & corona_mv_total> 54.5619 & salt<= 0.2 & conc<= 21.25 & mon_core_mw> 143.9765 & mon_corona_mw<= 287.115 & conc<= 16.5   
2         mon_corona_apol<= 28.3883 & ratio_vol<= 0.2213 & clogp_corona<= 0.73 & clogp_core<= 1.4 & mon_core_psa<= 47.1448 & dp_corona<= 64.5      
3         dp_corona> 27.5 & mon_corona_apol<= 58.2307 & clogp_core<= 1.18 & dp_corona<= 62.5                                                       
4         core_mw_total> 5330.96 & conc> 6.25 & charged<= 0.5 & mon_core_mw<= 144.4899                                                             

Phase                                \
Property  type      coef importance   
0         rule -3.807906  1.211516    
1         rule  2.655729  1.194534    
2         rule -2.554682  0.980607    
3         rule -1.460056  0.730022    
4         rule -1.497569  0.714774    

Phase                                                                                      Worm  \
Property                                                                                   rule   
0         ratio_vol> 0.2138 & ratio_vol> 0.2161                                                   
1         clogp_corona<= 0.25 & mon_core_psa<= 46.7805 & conc<= 11.25 & clogp_core<= 1.37         
2         sphere<= 0.5 & mon_core_psa<= 46.7805 & ratio_mass<= 0.2346 & core_mw_total> 32345.96   
3         mon_core_mw> 144.0775 & sphere> 0.5                                                     
4         dp_corona<= 77.5 & ratio_vol<= 0.2379 & mon_core_psa<= 46.7729 & dp_corona<= 61.0       

Phase                                \
Property  type      coef importance   
0         rule -1.774555  0.882646    
1         rule -1.626521  0.749633    
2         rule -1.639862  0.655945    
3         rule -1.276363  0.637751    
4         rule -1.239020  0.603698    

Phase                                                                                                                                                                                         Vesicle  \
Property                                                                                                                                                                                         rule   
0         ratio_vol<= 0.2604 & mon_corona_mv> 0.7838 & ratio_mass> 0.0443 & sphere<= 0.5 & temp> 53.0 & corona_mw_total<= 14063.4102 & ratio_mass<= 0.3571 & mon_core_apol> 20.5139 & dp_corona> 30.0   
1         mon_core_psa<= 46.5755 & sphere> 0.5 & conc> 7.5                                                                                                                                              
2         clogp_core<= 1.18 & ratio_vol<= 0.2422 & mon_corona_psa> 65.0 & corona_mw_total<= 12760.5298 & core_mv_total<= 1362.75                                                                        
3         corona_mw_total> 2502.955 & core_mw_total<= 57830.3594 & ratio_mass<= 0.2483 & core_mv_total> 258.0855 & dp_corona> 29.5                                                                      
4         corona_mw_total> 3982.275 & corona_mw_total<= 5806.375 & ratio_vol<= 0.3723                                                                                                                   

Phase                                
Property  type      coef importance  
0         rule  2.672133  1.196556   
1         rule -1.597402  0.796776   
2         rule  1.761069  0.761525   
3         rule  1.331125  0.569161   
4         rule  1.617735  0.559271

In [10]:
# get top 5:
sphere_top5_rule = rule_sphere.iloc[:5, ]
worm_top5_rule = rule_worm.iloc[:5, ]
vesicle_top5_rule = rule_vesicle.iloc[:5, ]

sphere_top5_rule.reset_index(drop=True, inplace=True)
worm_top5_rule.reset_index(drop=True, inplace=True)
vesicle_top5_rule.reset_index(drop=True, inplace=True)

top5_rule = pd.concat([sphere_top5_rule, worm_top5_rule, vesicle_top5_rule], axis=1)
upper_columns = ['Sphere', 'Worm', 'Vesicle']
lower_columns = ['rule', 'type', 'coef', 'importance']

top5_rule.columns = pd.MultiIndex.from_product([upper_columns, lower_columns], names=['Phase', 'Property'])
top5_rule.to_csv(os.path.join(OUTPUTPATH, 'Combine_Top5_Rules.csv'), index=False)
top5_rule


Phase                                                                                                           Sphere  \
Property                                                                                                          rule   
0         mon_core_psa<= 46.5755 & core_mv_total> 139.1655 & corona_mv_total> 77.7313                                    
1         corona_mw_total> 6338.1602 & mon_corona_psa> 58.59 & mon_core_psa> 36.415 & conc> 15.5 & ratio_mass<= 0.2884   
2         salt<= 0.2 & charged> 0.5 & ratio_vol<= 0.6135 & mon_core_mw<= 144.8274 & conc<= 16.1                          
3         conc> 16.1 & corona_mw_total<= 12760.5298 & mon_core_mw<= 145.3096 & ratio_mass<= 0.2867                       
4         mon_corona_apol<= 26.9812 & ratio_mass<= 0.2165 & clogp_core<= 1.37 & dp_corona<= 64.5                         

Phase                                \
Property  type      coef importance   
0         rule  1.746359  0.774758    
1         rule -1.671363  0.629908    
2         rule  1.481886  0.568818    
3         rule -1.349562  0.539825    
4         rule -1.584349  0.533887    

Phase                                                                                                               Worm  \
Property                                                                                                            rule   
0         dp_corona> 77.5 & core_mv_total<= 385.125 & core_mw_total> 23773.2002 & ratio_vol> 0.1063 & ratio_vol<= 0.2373   
1         mon_core_mw> 144.0775 & sphere> 0.5 & core_mw_total<= 32778.1992                                                 
2         mon_core_psa<= 46.7805 & mon_corona_psa<= 93.29                                                                  
3         corona_mw_total> 8461.2002 & core_mv_total<= 327.8658 & ratio_mass<= 0.3338 & mon_core_mw> 136.06                
4         corona_mv_total> 35.1238 & corona_mw_total<= 5985.855 & ratio_mass> 0.1459                                       

Phase                                \
Property  type      coef importance   
0         rule  3.178809  0.829368    
1         rule -1.572097  0.683494    
2         rule -1.793332  0.627536    
3         rule  1.484832  0.598430    
4         rule  1.678062  0.507953    

Phase                                                                                                                   Vesicle  \
Property                                                                                                                   rule   
0         clogp_core<= 1.1156 & mon_core_mv> 1.1145 & ratio_vol> 0.0454 & worm<= 0.5 & sphere<= 0.5 & corona_mw_total> 2972.075   
1         conc> 7.5 & ph<= 4.25 & corona_mw_total<= 5806.375 & ratio_vol<= 0.3723                                                 
2         mon_core_psa<= 46.7729 & corona_mw_total<= 5806.375 & corona_mw_total> 3982.275                                         
3         conc<= 21.25 & mon_core_psa<= 47.1448                                                                                   
4         conc> 7.5 & mon_core_psa<= 46.5755 & sphere> 0.5                                                                        

Phase                                
Property  type      coef importance  
0         rule  2.229113  0.798176   
1         rule  2.262592  0.708517   
2         rule  1.191152  0.457220   
3         rule -1.356461  0.450951   
4         rule -0.883557  0.440714